In [1]:
import sys
sys.path.append("..")

from utils.evaluation import *
from utils.utils import *

from data import dataset_preprocessing

from utils.evaluation import get_metrics
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso


from scipy import stats

import pandas as pd
import numpy as np
import os

import pickle

In [2]:
dataset_name = "cortez"
mode="cv"
RS=68
hct=10
test_ratio=0.2
val_ratio=0.1
folds=5
target = "continuous"
experiment_name = "EDM_results"

### Describe raw data

In [3]:
import arff # make sure to pip install liac-arff



df_mat = pd.read_csv(f"../data/raw/{dataset_name}/student-mat.csv",sep=";")
df_por = pd.read_csv(f"../data/raw/{dataset_name}/student-por.csv",sep=";")

df_mat["subject"] = "mat"
df_por["subject"] = "por"

df = pd.concat([df_mat,df_por])

In [4]:
y_col = "G3"
demographic_cols = ["sex", "age", "school", "address", "famsize", "famrel", "Pstatus", "Medu", "Fedu", "Mjob", "Fjob","reason","guardian", "traveltime", "famsup","paid","internet"]
perf_cols = ["G1","G2", "failures"]
activity_cols = ["studytime", "schoolsup", 'activities', 'nursery', 'higher', 'absences']
other_cols = ["subject", "Dalc", "Walc", "health", 'freetime', 'goout', 'romantic']
cat_cols = list(set(df.columns[np.logical_and(df.dtypes == "object", df.nunique()>2)]) - set([y_col]))

set(df.columns)-set([y_col]+demographic_cols+perf_cols+activity_cols+other_cols)

set()

In [5]:
desc_df_dict = {"No. of samples": df.shape[0],
           "No. of features": df.shape[1],
           "Performance features": len(perf_cols),
           "Demographic features": len(demographic_cols),
           "Activity features": len(activity_cols),
           "Other features": len(other_cols),
           "Categorical features": len(df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]),     
           "Total cardinality": df[df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]].nunique().sum(),     
           "% NA": df.isna().sum().sum()/sum(df.shape),
           "Target $\textbf{y} \in$": f"[1..{df[y_col].nunique()}]",
#            "High cardinality levels":  list(df.loc[:,list(df.columns[list(np.logical_and(df.nunique() >= 10, df.dtypes == "object"))])].nunique().sort_values().values),
          
}
desc_df = pd.DataFrame([desc_df_dict],index=["cortez"])
desc_df

,No. of samples,No. of features,Performance features,Demographic features,Activity features,Other features,Categorical features,Total cardinality,% NA,Target $\textbf{y} \in$
cortez,1044,34,3,17,6,7,4,17,0.0,[1..19]


In [6]:
print(desc_df.transpose().to_latex())

\begin{tabular}{ll}
\toprule
{} &   cortez \\
\midrule
No. of samples          &     1044 \\
No. of features         &       34 \\
Performance features    &        3 \\
Demographic features    &       17 \\
Activity features       &        6 \\
Other features          &        7 \\
Categorical features    &        4 \\
Total cardinality       &       17 \\
\% NA                    &        0 \\
Target \$\textbackslash textbf\{y\} \textbackslash in\$ &  [1..19] \\
\bottomrule
\end{tabular}



### Preprocessing and preparation

In [7]:
data_path = f"{mode}_RS{RS}_hct{hct}"
if mode == "cv":
    data_path += f"_{folds}folds"
elif mode == "train_test":
    data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
elif mode == "train_val_test":
    data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"


# If no data_dict for the configuration exists, run preprocessing, else load data_dict
if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
    dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
        data_dict = pickle.load(handle)


## Evaluation of categorical data treatment methods

In [8]:
conditions = ["ignore", "ohe", "target", "ordinal", "catboost", "glmm"]

In [9]:
early_stopping_rounds = 10
max_evals = 50

In [10]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle"):

    results_encodings = {}
    results_encodings_feature_importances = {}

    for fold in range(folds):
        target_scaler = data_dict[f"target_scaler_{fold}"]
        results_encodings[fold] = {}
        results_encodings_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = target_scaler.inverse_transform(data_dict[f"y_test_{fold}"].reshape(-1,1)).ravel()
        y_train_val = target_scaler.inverse_transform(np.concatenate([y_train,y_val]).reshape(-1,1)).ravel()

        y_train_val_pred_base = np.ones(y_train_val.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)
        y_test_pred_base = np.ones(y_test.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)

        results_encodings[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_encodings[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_encodings[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]
        results_encodings[fold]["Baseline"]["RMSE Test"] = np.sqrt(results_encodings[fold]["Baseline"]["MSE Test"])
        results_encodings[fold]["Baseline"]["RMSE Train"] = np.sqrt(results_encodings[fold]["Baseline"]["MSE Train"])


        for condition in conditions:
            print(f"Preparing results for fold {fold}, condition={condition}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]

    #         Define condition data subset
            if condition != "ignore":
                z_encoded_train = data_dict[f"z_{condition}_encoded_train_{fold}"] 
                z_encoded_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
                z_encoded_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

                X_train = pd.concat([X_train,z_encoded_train],axis=1)
                X_val = pd.concat([X_val,z_encoded_val],axis=1)
                X_test = pd.concat([X_test,z_encoded_test],axis=1)

            X_train_val = pd.concat([X_train,X_val])
            y_train_val = np.concatenate([y_train,y_val])

            # Train base models
            res, feats = evaluate_lr(X_train_val, y_train_val, X_test, y_test, target=target,tune=False, seed=RS, target_scaler=target_scaler)
            results_encodings[fold]["LR_"+condition] = res
            results_encodings_feature_importances[fold]["LR_"+condition] = feats
            results_encodings[fold]["LR_"+condition]["RMSE Test"] = np.sqrt(results_encodings[fold]["LR_"+condition]["MSE Test"])
            results_encodings[fold]["LR_"+condition]["RMSE Train"] = np.sqrt(results_encodings[fold]["LR_"+condition]["MSE Train"])

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS, target_scaler=target_scaler)
            results_encodings[fold]["XGB_"+condition] = res
            results_encodings_feature_importances[fold]["XGB_"+condition] = feats
            results_encodings[fold]["XGB_"+condition]["RMSE Test"] = np.sqrt(results_encodings[fold]["XGB_"+condition]["MSE Test"])
            results_encodings[fold]["XGB_"+condition]["RMSE Train"] = np.sqrt(results_encodings[fold]["XGB_"+condition]["MSE Train"])

            # Train tuned models
            res, feats = evaluate_lr(X_train_val, y_train_val, X_test, y_test, target=target, max_evals=max_evals, tune=True, seed=RS, target_scaler=target_scaler)
            results_encodings[fold]["LR_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["LR_"+condition+"_tuned"] = feats
            results_encodings[fold]["LR_"+condition+"_tuned"]["RMSE Test"] = np.sqrt(results_encodings[fold]["LR_"+condition+"_tuned"]["MSE Test"])
            results_encodings[fold]["LR_"+condition+"_tuned"]["RMSE Train"] = np.sqrt(results_encodings[fold]["LR_"+condition+"_tuned"]["MSE Train"])

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS, target_scaler=target_scaler)
            results_encodings[fold]["XGB_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["XGB_"+condition+"_tuned"] = feats
            results_encodings[fold]["XGB_"+condition+"_tuned"]["RMSE Test"] = np.sqrt(results_encodings[fold]["XGB_"+condition+"_tuned"]["MSE Test"])
            results_encodings[fold]["XGB_"+condition+"_tuned"]["RMSE Train"] = np.sqrt(results_encodings[fold]["XGB_"+condition+"_tuned"]["MSE Train"])
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'wb') as handle:
        pickle.dump(results_encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_encodings_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'rb') as handle:
        results_encodings = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'rb') as handle:
        results_encodings_feature_importances = pickle.load(handle)
    for fold in range(folds):
        target_scaler = data_dict[f"target_scaler_{fold}"]
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = target_scaler.inverse_transform(data_dict[f"y_test_{fold}"].reshape(-1,1)).ravel()
        y_train_val = target_scaler.inverse_transform(np.concatenate([y_train,y_val]).reshape(-1,1)).ravel()

        y_train_val_pred_base = np.ones(y_train_val.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)
        y_test_pred_base = np.ones(y_test.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)

        results_encodings[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_encodings[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_encodings[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]
        results_encodings[fold]["Baseline"]["RMSE Test"] = np.sqrt(results_encodings[fold]["Baseline"]["MSE Test"])
        results_encodings[fold]["Baseline"]["RMSE Train"] = np.sqrt(results_encodings[fold]["Baseline"]["MSE Train"])
        
        
results_encodings_df = pd.DataFrame(results_encodings[0]).transpose().sort_values("MSE Test",ascending=False).round(4)
results_encodings_df[["RMSE Train", "MSE Train", "R2 Train", "RMSE Test", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,RMSE Train,MSE Train,R2 Train,RMSE Test,MSE Test,R2 Test
Baseline,3.850500,14.826700,-0.000000,3.918300,15.352800,-0.013400
XGB_target,0.049800,0.002500,0.999800,1.725600,2.977700,0.803500
XGB_ohe,0.042700,0.001800,0.999900,1.693500,2.868100,0.810700
XGB_ordinal,0.045800,0.002100,0.999900,1.673300,2.799800,0.815200
XGB_ignore,0.054100,0.002900,0.999800,1.647400,2.714000,0.820900
XGB_catboost,0.023900,0.000600,1.000000,1.642400,2.697400,0.822000
XGB_glmm,0.029400,0.000900,0.999900,1.642000,2.696100,0.822000
LR_ohe_tuned,1.532000,2.347100,0.841700,1.610800,2.594700,0.828700
LR_glmm_tuned,1.526100,2.328900,0.842900,1.603400,2.571000,0.830300
LR_ignore_tuned,1.524000,2.322700,0.843300,1.601300,2.564100,0.830800


In [11]:
results_encodings_df = pd.DataFrame(results_encodings[1]).transpose().sort_values("MSE Test",ascending=False).round(4)
results_encodings_df[["RMSE Train", "MSE Train", "R2 Train", "RMSE Test", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,RMSE Train,MSE Train,R2 Train,RMSE Test,MSE Test,R2 Test
Baseline,3.822300,14.610200,-0.000000,4.022300,16.178500,-0.001800
XGB_catboost,0.026500,0.000700,1.000000,1.966100,3.865700,0.760600
LR_catboost,1.479300,2.188400,0.850200,1.724900,2.975100,0.815800
LR_ignore,1.484700,2.204300,0.849100,1.697600,2.882000,0.821500
LR_target,1.484700,2.204300,0.849100,1.697600,2.882000,0.821500
LR_glmm,1.484200,2.202900,0.849200,1.697400,2.881000,0.821600
LR_ordinal,1.482400,2.197500,0.849600,1.696500,2.878000,0.821800
LR_ohe,1.478200,2.184900,0.850400,1.695700,2.875400,0.822000
LR_ignore_tuned,1.497600,2.242800,0.846500,1.662200,2.763000,0.828900
LR_ordinal_tuned,1.497900,2.243600,0.846400,1.658700,2.751200,0.829600


### Performance Comparison

In [12]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "LR" in i)]
metric = "RMSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_ignore_tuned,LR_ohe_tuned,LR_target_tuned,LR_ordinal_tuned,LR_catboost_tuned,LR_glmm_tuned
0,3.862 (0.166),1.553 (0.103),1.552 (0.102),1.55 (0.101),1.553 (0.103),1.552 (0.102),1.551 (0.103)


In [13]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "RMSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(3).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])
    
def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_ignore_tuned,XGB_ohe_tuned,XGB_target_tuned,XGB_ordinal_tuned,XGB_catboost_tuned,XGB_glmm_tuned
0,3.862 (0.166),1.513 (0.116),1.513 (0.075),1.464 (0.064),1.518 (0.064),1.573 (0.078),1.498 (0.056)


In [14]:
res_df_lr.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_encodings = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_encodings.index = ["LR", "XGB"]
latex_df_encodings

,Baseline,ignore,ohe,target,ordinal,catboost,glmm
LR,3.862 (0.166),1.553 (0.103),1.552 (0.102),1.55 (0.101),1.553 (0.103),1.552 (0.102),1.551 (0.103)
XGB,3.862 (0.166),1.513 (0.116),1.513 (0.075),1.464 (0.064),1.518 (0.064),1.573 (0.078),1.498 (0.056)


In [15]:
print(latex_df_encodings.round(2).to_latex())


\begin{tabular}{llllllll}
\toprule
{} &       Baseline &         ignore &            ohe &         target &        ordinal &       catboost &           glmm \\
\midrule
LR  &  3.862 (0.166) &  1.553 (0.103) &  1.552 (0.102) &   1.55 (0.101) &  1.553 (0.103) &  1.552 (0.102) &  1.551 (0.103) \\
XGB &  3.862 (0.166) &  1.513 (0.116) &  1.513 (0.075) &  1.464 (0.064) &  1.518 (0.064) &  1.573 (0.078) &  1.498 (0.056) \\
\bottomrule
\end{tabular}



### Data subset comparisons

As it does not matter which encoding method is used we use 5CV-GLMM encoding for LR and Ordinal encoding for XGB

In [16]:
subsets = {"demo_only": demographic_cols,
           "perfact_only": perf_cols+activity_cols,
           "perfact_and_demo": perf_cols+activity_cols+demographic_cols,
           "all": list(df.columns)
          }

In [17]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle"):

    results_subsets = {}
    results_subsets_feature_importances = {}

    for fold in range(folds):
        target_scaler = data_dict[f"target_scaler_{fold}"]
        results_subsets[fold] = {}
        results_subsets_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = target_scaler.inverse_transform(data_dict[f"y_test_{fold}"].reshape(-1,1)).ravel()
        y_train_val = target_scaler.inverse_transform(np.concatenate([y_train,y_val]).reshape(-1,1)).ravel()

        y_train_val_pred_base = np.ones(y_train_val.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)
        y_test_pred_base = np.ones(y_test.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)

        results_subsets[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_subsets[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_subsets[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]
        results_subsets[fold]["Baseline"]["RMSE Test"] = np.sqrt(results_subsets[fold]["Baseline"]["MSE Test"])
        results_subsets[fold]["Baseline"]["RMSE Train"] = np.sqrt(results_subsets[fold]["Baseline"]["MSE Train"])


        for subset_key in subsets:
            print(f"Preparing results for fold {fold}, subset={subset_key}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]
        
            y_train_val = np.concatenate([y_train,y_val])

            # Define data subset for LR
            z_glmm_encoded_train = data_dict[f"z_glmm_encoded_train_{fold}"] 
            z_glmm_encoded_val = data_dict[f"z_glmm_encoded_val_{fold}"] 
            z_glmm_encoded_test = data_dict[f"z_glmm_encoded_test_{fold}"] 
            
            X_train_lr = pd.concat([X_train,z_glmm_encoded_train],axis=1)
            X_val_lr = pd.concat([X_val,z_glmm_encoded_val],axis=1)
            X_test_lr = pd.concat([X_test,z_glmm_encoded_test],axis=1)      
            X_train_val_lr = pd.concat([X_train_lr,X_val_lr])

            # Rescale GLMM
            for col in z_glmm_encoded_train.columns:
                z_mean = X_train_val_lr[col].mean()
                z_std = X_train_val_lr[col].std()
            
                X_train_val_lr[col] = (X_train_val_lr[col]-z_mean)/z_std
                X_test_lr[col] = (X_test_lr[col]-z_mean)/z_std
            
            
            # Define data subset for XGB
            z_ordinal_encoded_train = data_dict[f"z_ordinal_encoded_train_{fold}"] 
            z_ordinal_encoded_val = data_dict[f"z_ordinal_encoded_val_{fold}"] 
            z_ordinal_encoded_test = data_dict[f"z_ordinal_encoded_test_{fold}"] 
            X_train_xgb = pd.concat([X_train,z_ordinal_encoded_train],axis=1)
            X_val_xgb = pd.concat([X_val,z_ordinal_encoded_val],axis=1)
            X_test_xgb = pd.concat([X_test,z_ordinal_encoded_test],axis=1)
            X_train_val_xgb = pd.concat([X_train_xgb,X_val_xgb])


            # Define data subset for evaluation
            X_train_val_lr = X_train_val_lr[[i for i in X_train_val_lr.columns if i in subsets[subset_key]]]
            X_test_lr = X_test_lr[[i for i in X_test_lr.columns if i in subsets[subset_key]]]
            X_train_val_xgb = X_train_val_xgb[[i for i in X_train_val_xgb.columns if i in subsets[subset_key]]]
            X_test_xgb = X_test_xgb[[i for i in X_test_xgb.columns if i in subsets[subset_key]]]


            # Train base models
            res, feats = evaluate_lr(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target,tune=False, seed=RS, target_scaler=target_scaler)
            results_subsets[fold]["LR_"+subset_key] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key] = feats
            results_subsets[fold]["LR_"+subset_key]["RMSE Test"] = np.sqrt(results_subsets[fold]["LR_"+subset_key]["MSE Test"])
            results_subsets[fold]["LR_"+subset_key]["RMSE Train"] = np.sqrt(results_subsets[fold]["LR_"+subset_key]["MSE Train"])

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS, target_scaler=target_scaler)
            results_subsets[fold]["XGB_"+subset_key] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key] = feats
            results_subsets[fold]["XGB_"+subset_key]["RMSE Test"] = np.sqrt(results_subsets[fold]["XGB_"+subset_key]["MSE Test"])
            results_subsets[fold]["XGB_"+subset_key]["RMSE Train"] = np.sqrt(results_subsets[fold]["XGB_"+subset_key]["MSE Train"])

            # Train tuned models
            res, feats = evaluate_lr(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target, max_evals=max_evals, tune=True, seed=RS, target_scaler=target_scaler)
            results_subsets[fold]["LR_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key+"_tuned"] = feats
            results_subsets[fold]["LR_"+subset_key+"_tuned"]["RMSE Test"] = np.sqrt(results_subsets[fold]["LR_"+subset_key+"_tuned"]["MSE Test"])
            results_subsets[fold]["LR_"+subset_key+"_tuned"]["RMSE Train"] = np.sqrt(results_subsets[fold]["LR_"+subset_key+"_tuned"]["MSE Train"])

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS, target_scaler=target_scaler)
            results_subsets[fold]["XGB_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key+"_tuned"] = feats
            results_subsets[fold]["XGB_"+subset_key+"_tuned"]["RMSE Test"] = np.sqrt(results_subsets[fold]["XGB_"+subset_key+"_tuned"]["MSE Test"])
            results_subsets[fold]["XGB_"+subset_key+"_tuned"]["RMSE Train"] = np.sqrt(results_subsets[fold]["XGB_"+subset_key+"_tuned"]["MSE Train"])
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'wb') as handle:
        pickle.dump(results_subsets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_subsets_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'rb') as handle:
        results_subsets = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'rb') as handle:
        results_subsets_feature_importances = pickle.load(handle)
    for fold in range(folds):
        target_scaler = data_dict[f"target_scaler_{fold}"]
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = target_scaler.inverse_transform(data_dict[f"y_test_{fold}"].reshape(-1,1)).ravel()
        y_train_val = target_scaler.inverse_transform(np.concatenate([y_train,y_val]).reshape(-1,1)).ravel()

        y_train_val_pred_base = np.ones(y_train_val.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)
        y_test_pred_base = np.ones(y_test.shape[0])*target_scaler.mean_[0]#*np.mean(y_train_val)

        results_subsets[fold]["Baseline"] = {}
        eval_res_train = get_metrics(y_train_val, y_train_val_pred_base, target=target)
        for metric in eval_res_train.keys():
            results_subsets[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(y_test, y_test_pred_base, target=target)
        for metric in eval_res_test.keys():
            results_subsets[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]
        results_subsets[fold]["Baseline"]["RMSE Test"] = np.sqrt(results_subsets[fold]["Baseline"]["MSE Test"])
        results_subsets[fold]["Baseline"]["RMSE Train"] = np.sqrt(results_subsets[fold]["Baseline"]["MSE Train"])
        
        
results_subsets_df = pd.DataFrame(results_subsets[0]).transpose().sort_values("RMSE Test",ascending=False).round(4)
results_subsets_df[["RMSE Train", "MSE Train", "R2 Train", "RMSE Test", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,RMSE Train,MSE Train,R2 Train,RMSE Test,MSE Test,R2 Test
XGB_demo_only,1.367600,1.870400,0.873800,4.448700,19.790900,-0.306300
Baseline,3.850500,14.826700,-0.000000,3.918300,15.352800,-0.013400
LR_demo_only,3.672200,13.485000,0.090500,3.812200,14.533000,0.040700
XGB_demo_only_tuned,2.167900,4.699800,0.683000,3.799400,14.435600,0.047200
LR_demo_only_tuned,3.677700,13.525400,0.087800,3.795500,14.405800,0.049100
XGB_perfact_and_demo,0.082500,0.006800,0.999500,1.900100,3.610600,0.761700
XGB_all,0.045800,0.002100,0.999900,1.673300,2.799800,0.815200
XGB_perfact_only,0.533900,0.285000,0.980800,1.669200,2.786200,0.816100
LR_perfact_and_demo_tuned,1.557800,2.426800,0.836300,1.640700,2.691800,0.822300
LR_perfact_only_tuned,1.554800,2.417400,0.837000,1.632400,2.664800,0.824100


In [18]:
results_subsets_df = pd.DataFrame(results_subsets[1]).transpose().sort_values("RMSE Test",ascending=False).round(4)
results_subsets_df[["RMSE Train", "MSE Train", "R2 Train", "RMSE Test", "MSE Test", "R2 Test"]].style.highlight_min(subset=["MSE Train", "MSE Test"], color = 'lightgreen', axis = 0).highlight_max(subset=["R2 Train", "R2 Test"], color = 'lightgreen', axis = 0)

,RMSE Train,MSE Train,R2 Train,RMSE Test,MSE Test,R2 Test
XGB_demo_only,1.282100,1.643800,0.887500,4.437700,19.693100,-0.219400
Baseline,3.822300,14.610200,-0.000000,4.022300,16.178500,-0.001800
XGB_demo_only_tuned,2.479800,6.149600,0.579100,4.000500,16.004000,0.009000
LR_demo_only,3.635800,13.218800,0.095200,3.932500,15.464800,0.042400
LR_demo_only_tuned,3.647000,13.300500,0.089600,3.907700,15.270200,0.054500
XGB_perfact_only,0.528000,0.278800,0.980900,1.789300,3.201600,0.801800
XGB_perfact_and_demo_tuned,0.447800,0.200500,0.986300,1.785900,3.189400,0.802500
XGB_perfact_and_demo,0.078700,0.006200,0.999600,1.761600,3.103300,0.807800
LR_all,1.482700,2.198300,0.849500,1.698600,2.885300,0.821300
LR_perfact_and_demo,1.515200,2.295800,0.842900,1.687500,2.847800,0.823700


### Performance Results

In [19]:
# For LR
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "LR" in i)]
metric = "RMSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(2).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_demo_only_tuned,LR_perfact_only_tuned,LR_perfact_and_demo_tuned,LR_all_tuned
0,3.86 (0.166),3.76 (0.107),1.57 (0.099),1.58 (0.1),1.56 (0.101)


In [20]:
# For XGB
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "RMSE Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())*-1

df_mean = pd.DataFrame((-1*use_df).mean(axis=0).round(2).astype(str) + " (" + use_df.std(axis=0).round(3).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_demo_only_tuned,XGB_perfact_only_tuned,XGB_perfact_and_demo_tuned,XGB_all_tuned
0,3.86 (0.166),3.83 (0.149),1.49 (0.089),1.54 (0.138),1.54 (0.054)


In [21]:
res_df_lr.columns = [i[3:-6] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i[4:-6] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_subsets = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_subsets.index = ["LR", "XGB"]
latex_df_subsets

,Baseline,demo_only,perfact_only,perfact_and_demo,all
LR,3.86 (0.166),3.76 (0.107),1.57 (0.099),1.58 (0.1),1.56 (0.101)
XGB,3.86 (0.166),3.83 (0.149),1.49 (0.089),1.54 (0.138),1.54 (0.054)


In [22]:
print(latex_df_subsets.round(2).transpose().to_latex())


\begin{tabular}{lll}
\toprule
{} &            LR &           XGB \\
\midrule
Baseline         &  3.86 (0.166) &  3.86 (0.166) \\
demo\_only        &  3.76 (0.107) &  3.83 (0.149) \\
perfact\_only     &  1.57 (0.099) &  1.49 (0.089) \\
perfact\_and\_demo &    1.58 (0.1) &  1.54 (0.138) \\
all              &  1.56 (0.101) &  1.54 (0.054) \\
\bottomrule
\end{tabular}



### Feature Importance

In [23]:
# top_10_importances = {}

# for model in list(results_subsets_feature_importances[fold].keys()):
#     imp_df = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)

#     if "LR" in model:
#         direction = imp_df.apply(lambda x: np.sign(x))
#         imp_df = imp_df.abs()

#     imp_df = imp_df/imp_df.sum(axis=0)

#     mean_imp_df = imp_df.mean(axis=1)
#     std_imp_df = imp_df.std(axis=1)

#     mean_imp_df = mean_imp_df.sort_values(ascending=False)
#     std_imp_df = std_imp_df.loc[mean_imp_df.index]
#     final_imps = mean_imp_df[:10]
#     final_imps["Rest"] = sum(mean_imp_df[10:])
#     top_5_importances[model] = np.array([final_imps.index.values, final_imps.values])

In [24]:
demo_importances = {}
demo_importances_stds = {}

for model in list(results_subsets_feature_importances[fold].keys()):
    if "demo" in model or "all" in model:
        imp_df_all = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)
        
        if "LR" in model:
            direction = imp_df_all.apply(lambda x: np.sign(x))
            imp_df_all = imp_df_all.abs()
        if imp_df_all.sum().sum()!=0:
            imp_df = imp_df_all/imp_df_all.sum(axis=0)
        imp_df = imp_df.fillna(1/imp_df.shape[0])
#         imp_df = imp_df.loc[demographic_cols]

#         mean_imp_df = imp_df.mean(axis=1)
#         std_imp_df = imp_df.std(axis=1)

#         mean_imp_df = mean_imp_df.sort_values(ascending=False)
#         std_imp_df = std_imp_df.loc[mean_imp_df.index]
#         final_imps = mean_imp_df#[:10]
#         final_imps["Rest"] = sum(mean_imp_df[10:])
#         final_imps["Total"] = sum(mean_imp_df)
        demo_importances[model] = np.round(np.mean(imp_df.loc[demographic_cols].sum(axis=0)),2)#final_imps.values
        demo_importances_stds[model] = np.round(np.std(imp_df.loc[demographic_cols].sum(axis=0)),2)#final_imps.values


In [25]:
lr_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "LR" in i and "tuned" in i})
xgb_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "XGB" in i and "tuned" in i})
lr_demo_imp.index = [i[3:-6] for i in lr_demo_imp.index]    
xgb_demo_imp.index = [i[4:-6] for i in xgb_demo_imp.index]    

lr_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "LR" in i and "tuned" in i})
xgb_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "XGB" in i and "tuned" in i})
lr_demo_imp_stds.index = [i[3:-6] for i in lr_demo_imp_stds.index]    
xgb_demo_imp_stds.index = [i[4:-6] for i in xgb_demo_imp_stds.index]    


latex_df_imp = pd.DataFrame([lr_demo_imp.astype(str) + " (" + lr_demo_imp_stds.astype(str) + ")",
                             xgb_demo_imp.astype(str) + " (" + xgb_demo_imp_stds.astype(str) + ")"])
latex_df_imp.index = ["LR", "XGB"]
latex_df_imp

,demo_only,perfact_and_demo,all
LR,1.0 (0.0),0.08 (0.04),0.03 (0.02)
XGB,1.0 (0.0),0.26 (0.04),0.16 (0.04)


In [26]:
print(latex_df_imp.transpose().to_latex())

\begin{tabular}{lll}
\toprule
{} &           LR &          XGB \\
\midrule
demo\_only        &    1.0 (0.0) &    1.0 (0.0) \\
perfact\_and\_demo &  0.08 (0.04) &  0.26 (0.04) \\
all              &  0.03 (0.02) &  0.16 (0.04) \\
\bottomrule
\end{tabular}



### Mean impact of using demo features

In [27]:
np.random.seed(RS)
mean_abs_differences = []
for fold in range(folds):
    X_train = data_dict[f"X_train_{fold}"]
    y_train = data_dict[f"y_train_{fold}"]

    X_val = data_dict[f"X_val_{fold}"]
    y_val = data_dict[f"y_val_{fold}"]

    X_test = data_dict[f"X_test_{fold}"]
    y_test = data_dict[f"y_test_{fold}"]

    target_scaler = data_dict[f"target_scaler_{fold}"]

    y_train_val = np.concatenate([y_train,y_val])

    # Define data subset for LR
    z_glmm_encoded_train = data_dict[f"z_glmm_encoded_train_{fold}"] 
    z_glmm_encoded_val = data_dict[f"z_glmm_encoded_val_{fold}"] 
    z_glmm_encoded_test = data_dict[f"z_glmm_encoded_test_{fold}"] 

    X_train_lr = pd.concat([X_train,z_glmm_encoded_train],axis=1)
    X_val_lr = pd.concat([X_val,z_glmm_encoded_val],axis=1)
    X_test_lr = pd.concat([X_test,z_glmm_encoded_test],axis=1)      
    X_train_val_lr = pd.concat([X_train_lr,X_val_lr])

    # Rescale GLMM
    for col in z_glmm_encoded_train.columns:
        z_mean = X_train_val_lr[col].mean()
        z_std = X_train_val_lr[col].std()

        X_train_val_lr[col] = (X_train_val_lr[col]-z_mean)/z_std
        X_test_lr[col] = (X_test_lr[col]-z_mean)/z_std


    # Define data subset for XGB
    z_ordinal_encoded_train = data_dict[f"z_ordinal_encoded_train_{fold}"] 
    z_ordinal_encoded_val = data_dict[f"z_ordinal_encoded_val_{fold}"] 
    z_ordinal_encoded_test = data_dict[f"z_ordinal_encoded_test_{fold}"] 
    X_train_xgb = pd.concat([X_train,z_ordinal_encoded_train],axis=1)
    X_val_xgb = pd.concat([X_val,z_ordinal_encoded_val],axis=1)
    X_test_xgb = pd.concat([X_test,z_ordinal_encoded_test],axis=1)
    X_train_val_xgb = pd.concat([X_train_xgb,X_val_xgb])

    final_hyperparameters = tune_lasso(X_train_val_lr, y_train_val, max_evals=max_evals, seed=RS)
    lr = Lasso(alpha=final_hyperparameters["alpha"],
               random_state=RS)
#     lr = Lasso(alpha=0.001)
    lr.fit(X_train_val_lr,y_train_val)
    y_pred = target_scaler.inverse_transform(lr.predict(X_test_lr).reshape(-1,1)).ravel()

    is_not_demo = [i not in demographic_cols for i in X_train_val_lr.columns]
    y_pred_notdemo = target_scaler.inverse_transform(np.dot(X_test_lr.loc[:,is_not_demo],lr.coef_[is_not_demo]).reshape(-1,1)).ravel()
    mean_abs_diff = np.round(np.mean(np.abs(y_pred-y_pred_notdemo)),2)
    print(f"Mean absolute Difference w\o Demo: {mean_abs_diff}")
    print(f"RMSE Difference w\o Demo: {np.round(np.sqrt(np.mean(np.power(y_pred-y_pred_notdemo,2))),2)}")
    mean_abs_differences.append(mean_abs_diff)
    # is_demo = [i in demographic_cols for i in X_train_val_lr.columns]
    # y_pred_demo = target_scaler.inverse_transform(np.dot(X_test_lr.loc[:,is_demo],lr.coef_[is_demo]).reshape(-1,1)).ravel()

    # print(f"Mean absolute Difference with Demo: {np.mean(np.abs(y_pred-y_pred_demo))}")
    # print(f"RMSE Difference with Demo: {np.sqrt(np.mean(np.power(y_pred-y_pred_demo,2)))}")

SCORE: 0.33772816154692337                            
SCORE: 0.1603906219740407                             
SCORE: 0.16655529760918364                                                       
SCORE: 0.2626296144222785                                                       
SCORE: 0.23775787075615923                                                      
SCORE: 0.33467140692763786                                                      
SCORE: 0.23834428845620986                                                      
SCORE: 0.397071498986012                                                        
SCORE: 0.36275741001466816                                                      
SCORE: 0.25263909647371036                                                      
SCORE: 0.1726794165149927                                                        
SCORE: 0.3164141382542377                                                        
SCORE: 0.19054061381638826                                                   

SCORE: 0.2660191823530487                                                         
SCORE: 0.17139356495620234                                                        
SCORE: 0.18244313549402041                                                        
100%|██████████| 50/50 [00:03<00:00, 14.77trial/s, best loss: 0.16246038053880024]
The best hyperparameters are :  

{'alpha': 0.006185208905399513}
Mean absolute Difference w\o Demo: 0.37
RMSE Difference w\o Demo: 0.39
SCORE: 0.2852219206921866                             
SCORE: 0.3477436864125875                             
SCORE: 0.24064055653366676                                                      
SCORE: 0.17872346284965465                                                      
SCORE: 0.17539512542426353                                                       
SCORE: 0.3252347525810041                                                        
SCORE: 0.3577988519511469                                                        
SCORE: 0.1815

SCORE: 0.31643455804198883                                                        
SCORE: 0.1738386672833208                                                         
SCORE: 0.19520041884745185                                                        
SCORE: 0.2395806972387236                                                         
SCORE: 0.18705179664507093                                                        
SCORE: 0.16938833245915919                                                        
SCORE: 0.1771383144825181                                                         
SCORE: 0.2999503316721232                                                         
SCORE: 0.2713607721923562                                                         
100%|██████████| 50/50 [00:03<00:00, 15.23trial/s, best loss: 0.16312065485672392]
The best hyperparameters are :  

{'alpha': 0.018542617441564106}
Mean absolute Difference w\o Demo: 0.18
RMSE Difference w\o Demo: 0.18
SCORE: 0.3769241119787114        

In [28]:
np.mean(mean_abs_differences).round(2),np.std(mean_abs_differences).round(2)

(0.28, 0.06)